## 네이버 영화리뷰 텍스트 감성분석하기 💭

💬 영화리뷰 텍스트를 임베딩 레이어를 이용하여 감성 분석하는 프로젝트입니다.

### 1. 데이터 준비와 확인

In [5]:
import pandas as pd

# 데이터를 읽어봅시다. 
train_data = pd.read_table('~/aiffel/sentiment_classification/data/ratings_train.txt')
test_data = pd.read_table('~/aiffel/sentiment_classification/data/ratings_test.txt')


print(len(train_data), len(test_data))
train_data.head()

150000 50000


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


### 2. 데이터로더 구성

네이버 영화리뷰 데이터를 살펴보자.

먼저, 중복되는 값이 있는지 본다.

In [15]:
train_data['document'].nunique()

146182

3818개의 중복 데이터가 존재한다. -> **삭제해주어야 함!**

결측치가 있는지 확인해 보자.

In [14]:
for c in train_data.columns:
    print('{} : {}'.format(c, len(train_data.loc[pd.isnull(train_data[c]), c].values)))

id : 0
document : 5
label : 0


`document` 열에 5개의 `null`데이터가 있다. -> **삭제해주어야 함!**

In [30]:
from konlpy.tag import Mecab
import numpy as np
from collections import Counter

tokenizer = Mecab()
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

X_train = []
for x in train_data['document']:
    tokens = tokenizer.morphs(x)
    if tokens not in stopwords:
        X_train.append(tokens)
    
X_test = []
for x in test_data['document']:
    tokens = tokenizer.morphs(x)
    if tokens not in stopwords:
        X_train.append(tokens)
            
print(X_train[:10])

NotImplementedError: Wrong number or type of arguments for overloaded function 'Tagger_parse'.
  Possible C/C++ prototypes are:
    MeCab::Tagger::parse(MeCab::Model const &,MeCab::Lattice *)
    MeCab::Tagger::parse(MeCab::Lattice *) const
    MeCab::Tagger::parse(char const *)


In [26]:
from konlpy.tag import Mecab
import numpy as np
from collections import Counter

tokenizer = Mecab()
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

def load_data(train_data, test_data, num_words=num_words):
    
    # 데이터 중복 제거
    train_data.drop_duplicates(['document'])
    
    # 결측치 제거
    train_data = train_data.dropna(how = 'any')
    test_data = test_data.dropna(how = 'any')
    
    ## 토큰화(+ stopwords 제거)
    X_train = []
    for x in train_data['document']:
        tokens = tokenizer.morphs(x)
        if tokens not in stopwords:
            X_train.append(tokens)
    
    X_test = []
    for x in test_data['document']:
        tokens = tokenizer.morphs(x)
        if tokens not in stopwords:
            X_train.append(tokens)
    
        
    
    
    return X_train, y_train, X_test, y_test, word_to_index
    
    
    


NameError: name 'num_words' is not defined

In [3]:
index_to_word = {index:word for word, index in word_to_index.items()}

NameError: name 'word_to_index' is not defined

In [ ]:
# 문장 1개를 활용할 딕셔너리와 함께 주면, 단어 인덱스 리스트 벡터로 변환해 주는 함수입니다. 
# 단, 모든 문장은 <BOS>로 시작하는 것으로 합니다. 
def get_encoded_sentence(sentence, word_to_index):
    return [word_to_index['<BOS>']]+[word_to_index[word] if word in word_to_index else word_to_index['<UNK>'] for word in sentence.split()]

# 여러 개의 문장 리스트를 한꺼번에 단어 인덱스 리스트 벡터로 encode해 주는 함수입니다. 
def get_encoded_sentences(sentences, word_to_index):
    return [get_encoded_sentence(sentence, word_to_index) for sentence in sentences]

# 숫자 벡터로 encode된 문장을 원래대로 decode하는 함수입니다. 
def get_decoded_sentence(encoded_sentence, index_to_word):
    return ' '.join(index_to_word[index] if index in index_to_word else '<UNK>' for index in encoded_sentence[1:])  #[1:]를 통해 <BOS>를 제외

# 여러 개의 숫자 벡터로 encode된 문장을 한꺼번에 원래대로 decode하는 함수입니다. 
def get_decoded_sentences(encoded_sentences, index_to_word):
    return [get_decoded_sentence(encoded_sentence, index_to_word) for encoded_sentence in encoded_sentences]